In [ ]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
import requests as req
from tqdm import tqdm
import time
import os

In [ ]:
file_list = []
new_file_list = []
cnt = 1
U_A = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0"}

In [ ]:
driver = wb.Chrome()
driver.maximize_window()

# 금융통화위원회 의사록 페이지 별 첨부파일 href 수집
for i in range(1, 54) :
    min_url = f"https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={i}"
    driver.get(min_url)
    time.sleep(1)
    files = driver.find_elements(By.CSS_SELECTOR, "div.col.m2.s2.x3.fileLink ul > li > a")
    for i in range(len(files)) :
        file_list.append(files[i].get_attribute("href"))
driver.quit()

# file_list에서 불필요한 요소를 제거 한 new_file_list
for i in range(len(file_list)) :
    if "javascript" not in str(file_list[i]) and str(file_list[i])[-1] != "2" :
        new_file_list.append(file_list[i])
        
print(len(file_list))
print(len(new_file_list))
new_file_list

In [ ]:
# new_file_list 각 파일을 다운로드
for i in tqdm(new_file_list) :
    try :
        res = req.get(i, verify = False, timeout = 5)
        if res.status_code == 200 :
            # Content-Type 헤더 확인
            content_type = res.headers.get("Content-Type")
            print(content_type)
            
            # 확장자 분기 처리
            if content_type == "application/pdf" :
                file_extension = ".pdf"
            elif content_type == "application/x-hwp" :
                file_extension = ".hwp"
            else :
                file_extension = ".txt"
            file_name = f"minutes_{cnt}+{file_extension}"
            
            # 파일 경로 설정
            file_path = os.path.join('data', file_name)
            # 디렉토리 없을 경우 생성
            if not os.path.isdir("data") :
                os.mkdir("data")
                
            # 파일명 카운트 증가
            cnt += 1
            
            # 파일 열기, 쓰기
            open(file_path, "wb").write(res.content)
            print("Saved😇")
    except Exception as e :
        print("Error! : ", e)
        continue